In [24]:
from urllib.parse import urlparse

import allrank.models.losses as losses
import numpy as np
import os
import torch
from allrank.config import Config
from allrank.data.dataset_loading import load_libsvm_dataset, create_data_loaders
from allrank.models.model import make_model
from allrank.models.model_utils import get_torch_device, CustomDataParallel
from allrank.training.train_utils import fit
from allrank.utils.command_executor import execute_command
from allrank.utils.experiments import dump_experiment_result, assert_expected_metrics
from allrank.utils.file_utils import create_output_dirs, PathsContainer, copy_local_to_gs
from allrank.utils.ltr_logging import init_logger
from allrank.utils.python_utils import dummy_context_mgr
from argparse import ArgumentParser, Namespace
from attr import asdict
from functools import partial
from pprint import pformat
from torch import optim

In [2]:
import pickle

def load_dataset(path:str="dataset.pkl"):
    ds = None
    with open(path, "rb") as f:
        ds = pickle.Unpickler(f).load()
    return ds

dataset = load_dataset()

for train,val,test in dataset.dataset_generator(select_label='log_10'):
    break

In [ ]:
date = train.features_df.index.get_level_values('date').unique()

X = []
for d in date:
    X.append(train.features_df.xs(d, level='date').values)


200

6

In [15]:
from allrank.config import Config

config = Config.from_json('config.json')


In [16]:
from attr import asdict

import torch

from allrank.models.model import make_model
from allrank.models.model_utils import get_torch_device, CustomDataParallel



n_features =X[0].shape[1]
# gpu support
dev = get_torch_device()
print("Model training will execute on {}".format(dev.type))

# instantiate model
model = make_model(n_features=n_features, **asdict(config.model, recurse=False))
if torch.cuda.device_count() > 1:
    model = CustomDataParallel(model)
    print("Model training will be distributed to {} GPUs.".format(torch.cuda.device_count()))
model.to(dev)

Model training will execute on cuda


LTRModel(
  (input_layer): FCModel(
    (input_norm): Identity()
    (activation): Identity()
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): Linear(in_features=6, out_features=64, bias=True)
    )
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0-3): 4 x Linear(in_features=64, out_features=64, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=64, out_features=64, bias=True)
          (w_2): Linear(in_features=64, out_features=64, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (sublayer): ModuleList(
          (0-1): 2 x SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
      )
    )
    (norm): LayerNorm()
  )


In [23]:
from torch import optim

# load optimizer, loss and LR scheduler
optimizer = getattr(optim, config.optimizer.name)(params=model.parameters(), **config.optimizer.args)
loss_func = partial(getattr(losses, config.loss.name), **config.loss.args)
if config.lr_scheduler.name:
    scheduler = getattr(optim.lr_scheduler, config.lr_scheduler.name)(optimizer, **config.lr_scheduler.args)
else:
    scheduler = None

In [ ]:
with torch.autograd.detect_anomaly() if config.detect_anomaly else dummy_context_mgr():  # type: ignore
    # run training
    result = fit(
        model=model,
        loss_func=loss_func,
        optimizer=optimizer,
        scheduler=scheduler,
        train_dl=train_dl,
        valid_dl=val_dl,
        config=config,
        device=dev,
        output_dir=paths.output_dir,
        tensorboard_output_path=paths.tensorboard_output_path,
        **asdict(config.training)
    )